# 使用Bokeh观察二维函数

使用Bokeh的`image()`方法可以绘制二维图像，通过设置其`color_mapper`属性可以修改图像的颜色映射。实现和matplotlib的`imshow()`函数类似的效果。

In [1]:
import numpy as np
from bokeh.plotting import figure, Figure
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import widgetbox, row
output_notebook()

Loading BokehJS ...

下面显示二维正态分布的概率密度函数。

❶创建一个表示线性颜色映射表的`LinearColorMapper`对象，通过其`low`和`high`属性指定颜色映射表对应的最小值和最大值。

❷创建保存数据的`ColumnDataSource`对象，它的所有数据都是一个列表，列表中的每个元素对应一副图像。其中`img`保存图像的二维数据，`x`和`y`保存图像的左下角坐标，`dx`和`dy`保存图像的宽和高。

❸调用`image()`方法创建图像对象，通过`color_mapper`参数指定颜色映射表，`source`参数指定数据源。前5个参数为数据源中的列名。

❹创建表示颜色条的`ColorBar`对象，也通过其`color_mapper`参数指定颜色映射表。❺调用`fig.add_layout()`将颜色映射表放置到图表的右侧。

❻最后修改图表的X轴和Y轴的显示范围，以显示整个图像。

In [17]:
from scipy import stats
x0, x1, y0, y1 = -1, 1, -1, 1
Y, X = np.mgrid[x0:x1:50j, y0:y1:50j]
m = stats.multivariate_normal([0.0, 0], [[1.0, 0.1], [0.1, 1.0]])
Z = m.pdf(np.dstack([X, Y]))

cmap = LinearColorMapper("Viridis256", low=Z.min(), high=Z.max()) #❶
data = ColumnDataSource(data=dict(img=[Z], x=[x0], y=[y0], dx=[x1-x0], dy=[y1-y0])) #❷

fig = figure(plot_width=400, plot_height=380, toolbar_location="above")
fig.image("img", "x", "y", "dx", "dy", color_mapper=cmap, source=data) #❸
colorbar = ColorBar(color_mapper=cmap, label_standoff=12, border_line_color=None, location=(0,0))· #❹
fig.add_layout(colorbar, 'right') #❺
fig.x_range.update(start=x0, end=x1) #❻
fig.y_range.update(start=y0, end=y1)
show(fig)

Bokeh的图表绘制在浏览器中使用JavaScript实现，而数据的产生则由Python完成。有时候我们希望将整个Notebook保存为一个HTML文件，在脱离Python的环境下更新图表显示。这时就需要使用`CustomJS.from_py_func()`。它能将使用Python编写函数转换成JavaScript程序，然后使用各个对象的`js_on_change()`指定事件相应函数。

在下面的例子中，使用`Slider`控件修改概率密度函数的各个参数。当用户修改控件时，图像会即时更新。

❶由于数据由JavaScript计算产生，因此这里给`img`列指定的数据是一个宽和高为1的图像。并通过`nx`和`ny`列保存图像的实际大小。❷在JavaScript中使用`Float64Array()`创建实际大小的数组，注意这里只能创建一维数组。❸数组的形状信息保存在`source._shapes.img`中，将它设置为`nx`和`ny`。

❹在JavaScript中无法调用Python的运算函数，因此这里定义了一个嵌套函数`bivariate_normal()`用于计算图像个点的值。❺通过循环计算图像上个点的值之后，将新数组保存进`source.data.img`，并更新颜色映射表的`low`和`high`属性，❽最后调用`source.change.emit()`通知Bokeh数据已经更新，让Bokeh重回图表。


In [14]:
from bokeh.models.widgets import Slider

x0, x1, y0, y1 = -1, 1, -1, 1
slider_mux = Slider(start=-1, end=1, value=0, step=.1, title="mux")
slider_muy = Slider(start=-1, end=1, value=0, step=.1, title="muy")
slider_sigmax = Slider(start=0, end=2, value=1, step=.1, title="sigmax")
slider_sigmay = Slider(start=0, end=2, value=1, step=.1, title="sigmay")
slider_sigmaxy = Slider(start=-1, end=1, value=0, step=.1, title="sigmaxy")

sliders=[slider_mux, slider_muy, slider_sigmax, slider_sigmay, slider_sigmaxy]

widgets = widgetbox(sliders, width=400)

cmap = LinearColorMapper("Viridis256", low=0, high=1)
data = ColumnDataSource(data=dict(img=[np.zeros((1, 1))], x=[x0], y=[y0], dx=[x1-x0], dy=[y1-y0], nx=[50], ny=[50]), id="data") #❶

fig = figure(plot_width=400, plot_height=380, toolbar_location="above", id="fig")
fig.image("img", "x", "y", "dx", "dy", color_mapper=cmap, source=data, id="image")
colorbar = ColorBar(color_mapper=cmap, label_standoff=12, border_line_color=None, location=(0,0))
fig.add_layout(colorbar, 'right')
fig.x_range.update(start=x0, end=x1)
fig.y_range.update(start=y0, end=y1)

def callback(source=data, cmap=cmap, 
             mux=slider_mux, muy=slider_muy,
             sigmax=slider_sigmax, sigmay=slider_sigmay, sigmaxy=slider_sigmaxy):
    def bivariate_normal(X, Y, sigmax=1.0, sigmay=1.0,
                         mux=0.0, muy=0.0, sigmaxy=0.0): #❹
        Xmu = X-mux
        Ymu = Y-muy

        rho = sigmaxy/(sigmax*sigmay)
        z = Xmu**2/sigmax**2 + Ymu**2/sigmay**2 - 2*rho*Xmu*Ymu/(sigmax*sigmay)
        denom = 2*Math.PI*sigmax*sigmay*Math.sqrt(1-rho**2)
        return Math.exp(-z/(2*(1-rho**2))) / denom
    
    x0 = source.data.x[0]
    y0 = source.data.y[0]
    dx = source.data.dx[0]
    dy = source.data.dy[0]
    nx = source.data.nx[0]
    ny = source.data.ny[0]
    img = Float64Array(nx * ny) #❷
    
    i = 0
    _mux = mux.value
    _muy = muy.value
    _sigmax = sigmax.value
    _sigmay = sigmay.value
    _sigmaxy = sigmaxy.value
    for yi in range(ny): #❺
        for xi in range(nx):
            x = xi * dx / nx + x0
            y = yi * dy / ny + y0
            img[i] = bivariate_normal(x, y, _sigmax, _sigmay, _mux, _muy, _sigmaxy)
            i += 1
            
    source._shapes.img[0][0] = ny #❸
    source._shapes.img[0][1] = nx #❸
    source.data.img[0] = img #❻
    cmap.low = min(img) #❼
    cmap.high = max(img)
    source.change.emit() #❽
    
js_callback = CustomJS.from_py_func(callback)

for slider in sliders:
    slider.js_on_change("value", js_callback)

fig.js_on_change("inner_width", js_callback)
show(row(fig, widgets))